# Azure OpenAI Hackathon
**Team: CodeCrusher**

**Industry : Banking**.

**Sub LoB: – Retail Banking**.

**Problem Statement:** 
Reduction of underwriting efforts and increase TAT In Credit underwriting and Sanction of Small ticket loans in Retail Banking 
Solution: Open AI to perform Automated Credit underwriting and Auto sanction of small ticket loans for Retail Bank.   
Following parameters will be used in AI to provide Auto sanction.
- Credit Score
- Income 
- Deduction
- Company working 
- Spending pattern or Savings
- Purpose of loan

Model can be improved based on manual approval which can be show case for machine learning

.. For eg., working in IBM => Eligible for Auto sanction.  This will be fed into model to enable auto sanction for further applications. 

Outcome:
1. Instant Sanction of Loan 
2. Increase market share in personal loan and customer retention.
3. Quick TAT
4. Standardization of the process

In [24]:
import openai
import os
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity


# load environment variables/API key
# Store the API key in a .env file in the same directory as this notebook
load_dotenv() 
#Get open ai API key and endpoint from environment variables
API_KEY = os.getenv("AZURE_OPENAI_API_KEY") 
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 
#Build the connection to Azure OpenAI
openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"
model_engine_ada = "text-embedding-ada-002"  # This is the Deployment name in Azure
model_engine_davinci ="text-similarity-davinci-001" # This is the Deployment name in Azure

url = openai.api_base + "/openai/deployments?api-version=2022-12-01" 
r = requests.get(url, headers={"api-key": API_KEY})
#print to test if the connection is successful
#print(r.text)

**Data Source:**
https://www.kaggle.com/datasets/parisrohan/credit-score-classification

In [ ]:
# Load credit scoring data from CSV file
rawdata = pd.read_csv(os.path.join(os.getcwd(),'train.csv')) # Place the train.csv in the same directory you are running Jupyter Notebooks

#Get the required columns from the raw data
promptdata_df = rawdata[['Age','Occupation','AnnualIncome', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'NumofLoan', 'Type_of_Loan','Delay_from_due_date', 'NumofDelayedPayment', 'ChangedCreditLimit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt','Credit_Utilization_Ratio', 'Credit_History_Age','Payment_of_Min_Amount',
        'Total_EMI_per_month','Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance','Credit_Score']]

In [ ]:
# Generate prompts for each row in the dataframe
def generate_prompt_and_add_to_dataframe(df):
    prompts = []
    for index, row in df.iterrows():
        prompt = f"prompt: The person is {row['Age']} years old, working as a {row['Occupation']} with an annual income of {row['AnnualIncome']}, a monthly in-hand salary of {row['Monthly_Inhand_Salary']}, and has {row['Num_Bank_Accounts']} bank accounts and {row['Num_Credit_Card']} credit cards. They have an interest rate of {row['Interest_Rate']}% on their loans and have {row['NumofLoan']} outstanding loans, and has {row['Type_of_Loan']}. They have delayed their loan payments by {row['Delay_from_due_date']} days {row['NumofDelayedPayment']} times in the past year, but they have not changed their credit limit recently. They have made {row['Num_Credit_Inquiries']} credit inquiries in the last {row['Credit_History_Age']} months, have a {row['Credit_Mix']} credit mix of cards and loans, and have a total outstanding debt of {row['Outstanding_Debt']}. Their credit utilization ratio is {row['Credit_Utilization_Ratio']}%, and their credit history age is {row['Credit_History_Age']} years. They always pay the minimum amount due on their credit card bills, have a total EMI of {row['Total_EMI_per_month']} per month, and invest {row['Amount_invested_monthly']} per month. Their payment behavior is {row['Payment_Behaviour']}, and their monthly balance is {row['Monthly_Balance']}. Their credit score is currently {row['Credit_Score']}. , completion: {row['Credit_Score']}"
        prompts.append(prompt)
    df['Prompt'] = prompts
    return df

In [25]:
#Reduce the size of the data to n rows
credit_score_categories = ['Good', 'Standard', 'Poor']

# Create a new DataFrame to store the sampled data, this is to reduce the size of the data
sampled_data = pd.DataFrame()

# Sample the data for each credit score category
for category in credit_score_categories:
    # Filter the data for the current credit score category
    category_data = promptdata_df[promptdata_df['Credit_Score'] == category]
    
    # Sample the data for the current credit score category
    sampled_category_data = category_data.sample(n=1000)
    
    # concat the sampled data to the sampled_data DataFrame
    sampled_data = pd.concat([sampled_data, sampled_category_data])
    
 
# Generate prompts for each row in the sampled data
sampled_data = generate_prompt_and_add_to_dataframe(sampled_data)
#save the dataframe to a csv file "promptdata.csv"
#promptdata_df.to_csv('promptdata.csv', index=False)
sampled_data.count()


Age                         3000
Occupation                  3000
AnnualIncome                3000
Monthly_Inhand_Salary       3000
Num_Bank_Accounts           3000
Num_Credit_Card             3000
Interest_Rate               3000
NumofLoan                   3000
Type_of_Loan                2649
Delay_from_due_date         3000
NumofDelayedPayment         2803
ChangedCreditLimit          2940
Num_Credit_Inquiries        2943
Credit_Mix                  3000
Outstanding_Debt            3000
Credit_Utilization_Ratio    3000
Credit_History_Age          2742
Payment_of_Min_Amount       3000
Total_EMI_per_month         3000
Amount_invested_monthly     3000
Payment_Behaviour           3000
Monthly_Balance             3000
Credit_Score                3000
Prompt                      3000
dtype: int64

In [28]:
print(sampled_data['Prompt'].iloc[0])
#print and test the embedding using the first row of the dataframe
embedding = get_embedding(sampled_data['Prompt'].iloc[0], engine=model_engine_davinci)
#create a new dataframe with the embeddings and prompt
print(embedding)
#Test_df = pd.DataFrame({'Prompt': sampled_data['Prompt'].iloc[0], 'Embedding': embedding})
#Test_df.head(1)


prompt: The person is 15 years old, working as a Musician with an annual income of 18296.3, a monthly in-hand salary of 1672.691667, and has 8 bank accounts and 9 credit cards. They have an interest rate of 28% on their loans and have 9 outstanding loans, and has Debt Consolidation Loan, Mortgage Loan, Credit-Builder Loan, Auto Loan, Payday Loan, Student Loan, Student Loan, Payday Loan, and Credit-Builder Loan. They have delayed their loan payments by 46 days 22.0 times in the past year, but they have not changed their credit limit recently. They have made 11.0 credit inquiries in the last 11 Years and 7 Months months, have a Bad credit mix of cards and loans, and have a total outstanding debt of 3893.3. Their credit utilization ratio is 24.9458317%, and their credit history age is 11 Years and 7 Months years. They always pay the minimum amount due on their credit card bills, have a total EMI of 84.26469322 per month, and invest 44.41986101 per month. Their payment behavior is High_spe

In [5]:
# Get the embeddings for each prompt in the sampled data
#sampled_data['embedding'] = sampled_data['Prompt'].apply(lambda x: get_embedding(x, engine=model_engine_davinci))
#sampled_data.to_csv('CreditScore_embeddings_davinci.csv')

In [3]:
#Load the embeddings from the csv file
Search_df = pd.read_csv(os.path.join(os.getcwd(),'./data/CreditScore_embeddings_davinci.csv')) # Place the *.csv in the same directory you are running Jupyter Notebooks
#Convert the embeddings to numpy arrays
Search_df['embedding'] = Search_df['embedding'].apply(eval).apply(np.array)

In [21]:
#Get the Input from the user
search_term = input('Please provide the information to get a credit scoring prediction: ')

# Semantic Search

Now that we have our word embeddings stored, let's load them into a new dataframe and use it for semantic search. Since the 'embedding' in the CSV is stored as a string, we'll use apply() and to interpret this string as Python code and convert it to a numpy array so that we can perform calculations on it.

In [22]:
#Get the embedding for the search term
search_term_vector = get_embedding(search_term, engine=model_engine_davinci)
#search_term_vector

 Once we have a vector representing that word, we can see how similar it is to other words in our dataframe by calculating the cosine similarity of our search term's word vector to each word embedding in our dataframe.

In [23]:

#get the similarity score
Search_df['similarities'] = Search_df['embedding'].apply(lambda x: cosine_similarity(x, search_term_vector))
#Search_df.sort_values("similarities", ascending=False).head(10)

#print crdit score column only  for top 1 result
print('The predictive credit score is ' + Search_df.sort_values("similarities", ascending=False).head(1)['Credit_Score'])

#Print the top 10 results with the highest similarity score
Search_df.sort_values("similarities", ascending=False).head(10)[['similarities','Age','Occupation','AnnualIncome', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'NumofLoan', 'Type_of_Loan','Delay_from_due_date', 'NumofDelayedPayment', 'ChangedCreditLimit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt','Credit_Utilization_Ratio', 'Credit_History_Age','Payment_of_Min_Amount',
        'Total_EMI_per_month','Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance']]

781    The predictive credit score is Good
Name: Credit_Score, dtype: object


,similarities,Age,Occupation,AnnualIncome,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,NumofLoan,Type_of_Loan,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
781,0.803046,45,Engineer,24282.15,1875.512500,4,6,6,0,NaN,...,0.0,Good,923.75,26.157522,32 Years and 2 Months,No,0.000000,43.855790,High_spent_Medium_value_payments,393.6954598
8932,0.801554,45,Engineer,102261.27,8731.772500,3,5,13,1,Debt Consolidation Loan,...,2.0,Standard,1164.89,38.131621,20 Years and 7 Months,No,79.779796,0.000000,High_spent_Large_value_payments,861.1657265
9044,0.801254,45,Engineer,102261.27,8731.772500,3,5,13,1,Debt Consolidation Loan,...,2.0,Standard,1164.89,29.910533,20 Years and 10 Months,No,79.779796,0.000000,High_spent_Small_value_payments,643.8137504
162,0.799653,45,Developer,66571.10,5511.591667,2,1,2,3,"Credit-Builder Loan, Credit-Builder Loan, and ...",...,0.0,Good,1354.15,39.270062,16 Years and 0 Months,No,86.952032,185.406266,High_spent_Medium_value_payments,528.8008682
1237,0.799244,46,Scientist,121881.24,10126.770000,0,3,2,4,"Personal Loan, Home Equity Loan, Auto Loan, an...",...,3.0,Good,771.77,34.741512,NaN,No,388.600435,792.079192,Low_spent_Large_value_payments,101.9973726
6663,0.799161,45,Developer,85421.19,7079.432500,4,7,10,1,Auto Loan,...,0.0,Standard,1203.57,36.099478,17 Years and 1 Months,Yes,38.053440,446.148749,High_spent_Small_value_payments,483.7410613
6236,0.798964,45,Engineer,60226.16,5127.846667,4,4,5,0,NaN,...,2.0,Good,873.05,33.237295,NaN,No,31012.000000,114.148854,High_spent_Large_value_payments,638.6358131
8559,0.798877,45,Engineer,85862.60,7080.216667,3,7,13,0,NaN,...,4.0,Standard,1201.93,36.792612,16 Years and 9 Months,Yes,0.000000,363.346073,High_spent_Small_value_payments,604.6755938
7130,0.797850,45,Manager,145168.96,11910.413330,4,6,7,2,"Auto Loan, and Not Specified",...,7.0,Standard,127.56,37.401127,10 Years and 0 Months,Yes,141.229980,931.180187,Low_spent_Medium_value_payments,398.6311664
1849,0.797562,54,Engineer,73579.80,6200.650000,4,5,7,2,"Auto Loan, and Credit-Builder Loan",...,4.0,Good,1226.97,39.030677,24 Years and 10 Months,No,100.298695,322.394388,High_spent_Small_value_payments,457.3719175
